In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import lightning as L

In [2]:
token_to_id = { 'what' : 0, 'is' : 1, 'hockey' : 2, 'NHL' : 3, '<EOS>' : 4,}

In [3]:
id_to_token = dict(map(reversed, token_to_id.items()))

In [4]:
inputs = torch.tensor([[token_to_id["what"],
                       token_to_id["is"],
                       token_to_id["hockey"],
                       token_to_id["<EOS>"],
                       token_to_id["NHL"],]])

In [5]:
labels = torch.tensor([[token_to_id["is"],
                       token_to_id["what"],
                       token_to_id["<EOS>"],
                       token_to_id["NHL"],
                       token_to_id["<EOS>"],]])

In [6]:
dataset = TensorDataset(inputs,labels)
dataloader = DataLoader(dataset)

In [7]:
class PositionEncoding(nn.Module):
    def __init__(self, d_model=2, max_len=6):
        super().__init__()
        
        pe = torch.zeros(max_len, d_model)
        
        position = torch.arange(start=0, end=max_len, step=1).float().unsqueeze(1)
        embedding_index = torch.arange(start=0, end=d_model, step = 2).float()
        
        div_term = 1/torch.tensor(10000.0)**(embedding_index/d_model)
        
        pe[:,  0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe',pe)
        
    def forward(self, word_embeddings):
        
        return word_embeddings + self.pe[:word_embeddings.size(0), :]